# Project Final Report


# Testing

**Predicting Subscription 
Habits based on Age and Played Hours in a Minecraft Server**

Samantha Cham, Daha Choi, Trini Laughlen, and Madeline Miles

### Introduction


Today, we are going to be analyzing a dataset, from a research group in Computer Science at UBC, led by Frank Wood. This dataset contains information about players in a particular MineCraft Server, from the player's age to total number of hours played. Using this dataset, we are aiming to answer the following question: 

Can age and total played hours predict if the player is subscribed to a game-related news article?

To answer this question, we used the players.csv dataset:

*players.csv* (196 observations, 7 variables):  

•  *Experience Level* (factor): Categorizes players based on skill level (e.g., Beginner, Amateur, Pro).  
•  *Subscription Status* (logical): Indicates whether the player is subscribed to the newsletter (TRUE/FALSE).  
•  *Hashed Email* (string): Unique identifier for each player.  
•  *Total Hours Played* (double): The total number of hours the player has spent in-game.  
•  *First Name* (string): The player's in-game or registered first name.  
•  *Gender* (string): Self-reported gender identity.  
•  *Age* (integer): The player's age in years.  


*Summary Statistics:*
•  *Played Hours:* Mean = 5.85, Min = 0, Max = 223.1 
•  *Age:* Mean = 20.52, Min = 8, Max = 50 
•  *Subscription Rate:* 73.47% 
•  *Experience Level Distribution:* 17.86% Beginner, 32.14% Amateur, 18.37% Regular, 24.49% Veteran, 7.14% Pro 


*Some Issues we may deal with:*
•  Inconsistent variable naming (hashedEmail in camelCase vs. played_hours in snake_case).
•  Capitalization inconsistency (Age vs. gender).
•  Some categorical groups are very small.

### Methods


In [ ]:
#load libraries needed
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

#set the seed to make the process reproducible
set.seed(2025)

#read in data
players<-read_csv('https://raw.githubusercontent.com/madeline1312/toy_ds_project/refs/heads/main/players.csv')|>
mutate(subscribe=as_factor(subscribe))|>#made as a factor so we can build our classification model
select(Age, played_hours, subscribe)|># We can see NA's in the data, so we have to remove them to bind_cols later
drop_na()
players


In [ ]:
#find the summary statistics of what we are looking at so we know what our scales are for the predictors
players_stats<-players|>
mutate(mean_age=mean(Age,na.rm=TRUE))|>
mutate(min_age=min(Age,na.rm=TRUE))|>
mutate(max_age=max(Age, na.rm=TRUE))|>
mutate(min_played_hours=min(played_hours))|>
mutate(max_played_hours=max(played_hours))|>
mutate(mean_played_hours=mean(played_hours))|> 
select(mean_age, min_age, max_age, min_played_hours, max_played_hours, mean_played_hours)|> 
slice(1)
players_stats


In our analysis in finding a summary of the data set that is needed for our analysis we found that the minimum age of a player is 8 the max age is 50 and the mean age is 20.5. This shows that since we have such a variety of ages and a large age range we have to center and scale our data in order for the model to be accurate when making predictions.
The analysis found that the minmimum number of players hours is 0, while the max is 223.1 hours and the average is 5.9 hours. This tells us that since the range is so wide we have to center and scale our data so that it is given the same weight as the age of the player, when used to predict the subscribtion habits of a player. 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8) 
players_plot<-players|>
ggplot(aes(x=played_hours, y=Age, colour=subscribe))+
geom_point(alpha=0.8)+
labs(y="Player's Age", x="Total Played Hours", 
     colour="Subscribed to a game related newsletter?")+
ggtitle("Figure 1: Age, hours played, and subscription plot")+
scale_x_log10()+
theme(text = element_text(size = 12))

players_plot


This plot shows all three variables that we are considering together, and there appears to be no direct correlation between the amount of hours played on the server along with the players age to their subscrbtion habits to a game related newsletter. It can be seen that many of the played hours come from the younger players around our mean age, and that the players who are subscribed to a game-related newsletter often have more playing hours than the ones who aren't subscribed to a game related newsletter. We chose to make a scatter plot bevcause we are comparing two variables that are numeric so it is easiest to compare them on a scatterplot, and we used the colour to show our class of subscription to see if there was a correlation. 

We should look at each predcitor we will use to see if their is a correlation and find what we can expect the outcome to be from there. 

In [ ]:
Age_plot<-players|>
ggplot(aes(x=Age, fill=subscribe))+
geom_bar()+
labs(x="Player's Age", y="Portion of that age group subscribed to a newsletter", 
     fill="Subscribed to a game related newsletter?")+
ggtitle("Figure 2: Players Age vs Subscription")+
theme(text = element_text(size = 12))
Age_plot


This plot shows the players age vs their subscribtion habits to a game related newsletter. We can see a trend here that the younger a player is the more likely they are to subscribe to a game related newsletter, but there are a few outliers in this graph as well. We chose a bar graph because we are comparing a numerical variable (Age), and the percentage of people who are subscribed to a newsletter, which is represented by the colour. 

In [ ]:
#This plot shows hours played vs Subsription
played_plot<-players|>
ggplot(aes(x=played_hours, fill=subscribe))+
geom_histogram()+
labs(y="Number of players subscribed to a newsletter", x="Total Played Hours", 
     fill="Subscribed to a game related newsletter?")+
ggtitle("Figure 3: Hours Played vs Subscription")+
scale_x_log10()+
theme(text = element_text(size = 12))
played_plot


This plot shows that the more hours you play the more likely you are to subscribe to a game related newsletter, but there isn't a clear trend on the relationships between the number of hours played on the server and if the player is subscribed to a game related newsletter.  We chose a bar graph because we are comparing a numerical variable (Hours Played), and the percentage of people who are subscribed to a newsletter, which is represented by the colour. 

When building our model and later testing it, we do expect that the model is pretty accurate but not very accurate as their is a clear trend with the age of the player and if they are subscribed to a game-related newsletter, but not as strong of a relationship between the hours played and subscrbtion habits. Because one of the two relationships isn't extremly clear through visaulizations, we expect the model to be accurate as there is a clear relationship between one predictor and the outcome but not as clear between the second predictor and the outcome. 

In [ ]:
set.seed(2025)
#The seed ensures reproducability
#Now we will build out model
#Let's seperate the data into training and testing data sets
players_split<-players|>
initial_split(prop=0.6, strata=subscribe)
players_train<-training(players_split)
players_test<-testing(players_split)


In [ ]:
set.seed(2025)
#Build the model, initally use tune until the best k value is found. 
knn_tune<-nearest_neighbor(weight_func='rectangular',neighbors=tune())|>
set_engine('kknn')|>
set_mode('classification')

players_recipe<-recipe(subscribe~Age+played_hours, data=players_train)|>
step_center(all_predictors())|>
step_scale(all_predictors())

workflow<-workflow()|>
add_recipe(players_recipe)|>
add_model(knn_tune)

players_vfold<-vfold_cv(players_train, v=5, strata=subscribe)

players_table<-tibble(neighbors=seq(from=1, to=10, by=1))

knn_results<-workflow|>
tune_grid(resample=players_vfold, grid=players_table)|>
collect_metrics()|>
filter(.metric=='accuracy')

cross_val_plot<-knn_results|>
ggplot(aes(y=mean, x=neighbors))+
geom_point()+
geom_line()+
labs(y="mean classification accuracy", x="number of neighbours used in classification")+
ggtitle("Figure 4: K value vs mean accuracy")+
theme(text = element_text(size = 12))
cross_val_plot


This plot shows that with the testing data only the highest accuracy knn that we get from the cross-validation with 5 folds is 9, with an accuracy of just under 65%, this is good for the training set, and now we rebuild our model with a neighbors value of k=9. We chose this as a line plot and a scatterplot so we wee easily able to visulaize the best k value that gave us the highest accuracy on the training set we made earlier. 

In [ ]:
set.seed(2025)
#build the model with k=9 to have the best accuracy
players_knn<-nearest_neighbor(weight_func='rectangular', neighbors=9)|>
set_mode('classification')|>
set_engine('kknn')

players_workflow<-workflow()|>
add_recipe(players_recipe)|>
add_model(players_knn)|>
fit(data=players_train)


### Results


In [ ]:
players_results<-players_workflow|>
predict(players_test)|> 
bind_cols(players_test)
players_results


In [ ]:
players_results_plot<-players_results|>
ggplot(aes(x=.pred_class, fill=subscribe))+
geom_bar()+
xlab("Predicted Subscribtion habit based on Age and played hours")+
ylab("Number of people predicted with that habit")+
labs(fill="The person is subscribed to a game related newsletter")+
ggtitle("Figure 5: Prediction graph")+
theme(text = element_text(size = 12))
players_results_plot


This plot shows that the model predicted a person to subscribe to a newsletter more often than a person not subscribing to a newsletter, and while that prediction was accurate for many people it over predicted this value. When predicting that a person wouldn't subscribe to a game-related newsletter, the model predicted that the player wouldn't when most of the time they would, only a small number of predictions that were "FALSE" were in reality actually "FALSE". 

In [ ]:
set.seed(2025)
#test the model to see if it is accurate with the testing data
knn_accuracy<-players_results|>
metrics(truth=subscribe, estimate=.pred_class)|>
filter(.metric=="accuracy")|>
pull()

knn_accuracy


The accuracy of our model is 57.69%. This demonstrates that there is a slight correlation between the number of hours played and the age of player in relation to if they are subscribed to a game related newsletter, but their isn't a strong relationship between the two predictors and what we were predicting. 

### Discussion


In this project, we explored if data concerning a Minecraft server player's age and number of played hours is able to predict if they would be subscribed to the game-related newsletter. In our exploration, we found the age range of players to be from 8 to 50, with a mean of 20.5. Additionally, we discovered that the average number of hours played was 5.9, but there was a large range, with the maximum number of hours being 223.1. In our initial exploration, there appeared to be no direct correlation between the amount of hours played on the server along with the player's age to their subscription habits to a game-related newsletter. However, there was some correlation between age and subscription habits (younger players were more likely to subscribe). Initial graphs also suggested that players who subscribed had a higher number of hours played. 

Considering these initial trends, we used K-Nearest Neighbour classifcation to investigate this relationship further. Our cross-validation results suggest that the best K value to use was k=9, which provided a 65% classifcation accuracy (which is suffcient for a training set). We then evaluated the training set performance with the test set. The final test accuracy was 57.69%, as mentioned above. This suggests that there is a slight correlation between the number of hours played and the age of the player concerning whether they are subscribed to a game-related newsletter, but there isn't a strong relationship between the two predictors and what we were predicting. This does answer our question, because it demonstrates that there is some correlation between age, played hours and suscription habits, there in our model there was not significant correlation idetified. 

These results were somewhat expected because, based on our initial exploration showed a slight correlation between the individual predictors and if they were subscribed. The overall predictive accuracy was quite low, considering our earlier visualizations, as they indicated that the two variables are not reliable predictions. In the future, as age and hours played were not reliable predictors, we could explore additional features that identify a player's interest in the magazine in order to eventually make a model that could successfully predict magazine subscribers, making it easier to identify the magazine's target audience and create marketing strategies. 

There are some limitations to our model. The size of the data set is a limitation because when it is split there is a limited number of points. This causes the model to ber less accurate as the K-value is working off a limited number of values. This could be improved by having a larger data set from the minecraft server. This would our model because more data would lead to a more accurate prediction as the k value has more values to work off of, through this process a clear trend may establish itself. 